<a href="https://colab.research.google.com/github/GusevMihail/seamless_textute_generator/blob/master/batch_image_resizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import cv2
import albumentations as albu
import numpy as np


In [10]:
import os.path
import sys
if 'google' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir data
    !mkdir data/train
    if os.path.exists('data/train'):
        !cp /content/drive/MyDrive/Colab/seamless_textute_generator/data/concrete_maps.zip data
        !unzip -q -n data/concrete_maps.zip -d data/train

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/train’: File exists


In [11]:
data_folder = r'data/train'
image_names = tuple(f for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f)))

In [27]:
!mkdir data/resize
out_folder = r'data/resize'

## Пакетный ресайз

In [ ]:
%%time
MAX_SIZE = 1000

for f in os.listdir(data_folder):
    in_f = os.path.join(data_folder, f)
    if os.path.isfile(in_f):
        image = cv2.imread(in_f)
        transform = albu.LongestMaxSize(max_size=MAX_SIZE, interpolation=cv2.INTER_CUBIC)
        resized = transform(image=image)['image']
        out_f = os.path.join(out_folder, f)
        cv2.imwrite(out_f, resized)
    

CPU times: user 26.5 s, sys: 418 ms, total: 26.9 s
Wall time: 25.8 s


In [ ]:
for f in os.listdir(out_folder):
    f = os.path.join(out_folder, f)
    if os.path.isfile(f):
        print('READING RESIZED IMAGE')
        %time image = cv2.imread(f)
        break

READING RESIZED IMAGE
CPU times: user 14.2 ms, sys: 6 µs, total: 14.2 ms
Wall time: 15.7 ms


In [ ]:
!zip -j -r -q data/concrete_maps_1K.zip data/resize/* 

In [ ]:
!cp data/concrete_maps_1K.zip /content/drive/MyDrive/Colab/seamless_textute_generator/data

## "Плиточная" нарезка изображений

In [28]:
%%time

OUT_W = 1000
OUT_H = 1000

def split_image_side(in_side, out_side):
    return np.linspace(start=0,
                       stop=in_side - out_side,
                       num=in_side // out_side + 1,
                       dtype=np.int16)

for f in os.listdir(data_folder):
    in_f = os.path.join(data_folder, f)
    if os.path.isfile(in_f):
        image = cv2.imread(in_f)
        hight, width, channels = image.shape
        i = 0
        for h in split_image_side(hight, OUT_H):
            for w in split_image_side(width, OUT_W):
                i += 1
                resized = image[h:h + OUT_H, w:w + OUT_W]
                #TODO написать разбивку имени файла на имя и расширение,
                # чтобы добавлять суфикс к имени перед расширением.
                f_name, f_ext = os.path.splitext(f)
                out_f = os.path.join(out_folder, f'{f_name}_{i:0>3}{f_ext}')
                cv2.imwrite(out_f, resized)

CPU times: user 1min 5s, sys: 2.3 s, total: 1min 7s
Wall time: 1min 7s


In [29]:
!zip -j -r -q data/concrete_maps_1K.zip data/resize/* 

In [30]:
!cp data/concrete_maps_1K.zip /content/drive/MyDrive/Colab/seamless_textute_generator/data

## Далее следуют тесты


In [ ]:
!mkdir data/test
!unzip -q data/concrete_maps_1K.zip -d data/test

In [26]:
!rm -r data/resize

In [ ]:
!rm data/concrete_maps_1K.zip

In [ ]:
!man zip